<a href="https://colab.research.google.com/github/dakariakin/homeworks/blob/main/3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.feature_extraction import DictVectorizer

In [28]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv')
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [29]:
base = ['Make', 'Model', 'Year', 'Engine HP', 'Engine Cylinders', 'Transmission Type', 'Vehicle Style', 'highway MPG', 'city mpg', 'MSRP']
df = df[base]
df.head()

,Make,Model,Year,Engine HP,Engine Cylinders,Transmission Type,Vehicle Style,highway MPG,city mpg,MSRP
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500


In [30]:
df.columns = df.columns.str.replace(' ', '_').str.lower()
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,msrp
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500


In [31]:
df.isnull().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
msrp                  0
dtype: int64

In [32]:
df["engine_hp"] = df["engine_hp"].fillna(0)
df["engine_cylinders"] = df["engine_cylinders"].fillna(0)
df.isnull().sum()

make                 0
model                0
year                 0
engine_hp            0
engine_cylinders     0
transmission_type    0
vehicle_style        0
highway_mpg          0
city_mpg             0
msrp                 0
dtype: int64

In [33]:
df = df.rename(columns={'msrp': 'price'})

In [34]:
df["transmission_type"].value_counts(normalize=True)

AUTOMATIC           0.693806
MANUAL              0.246349
AUTOMATED_MANUAL    0.052543
DIRECT_DRIVE        0.005708
UNKNOWN             0.001595
Name: transmission_type, dtype: float64

In [35]:
numerical = ["year", "engine_hp", "engine_cylinders", "highway_mpg", "city_mpg", "price"]
categorical = ['make', 'model', 'transmission_type', 'vehicle_style']

In [36]:
df[numerical].corr()

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.000000,0.338714,-0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306,0.526274
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000,-0.157676
price,0.227590,0.650095,0.526274,-0.160043,-0.157676,1.000000


In [11]:
mean_price = df.price.mean()
mean_price

40594.737032063116

In [12]:
def set_above_average(ser):
  if ser.price > mean_price:
    return 1
  else:
    return 0

df["above_average"] = df.apply(set_above_average, axis=1)
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price,above_average
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135,1
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650,1
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350,0
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450,0
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500,0


In [13]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

df_full_train = df_full_train.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

df_train.shape, df_val.shape, df_test.shape

((7148, 11), (2383, 11), (2383, 11))

In [14]:
[
    round(mutual_info_score(df_full_train.make, df_full_train.above_average), 2),
    round(mutual_info_score(df_full_train.model, df_full_train.above_average), 2),
    round(mutual_info_score(df_full_train.transmission_type, df_full_train.above_average), 2),
    round(mutual_info_score(df_full_train.vehicle_style, df_full_train.above_average), 2)
]

[0.24, 0.46, 0.02, 0.08]

In [15]:
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

del df_train['above_average']
del df_val['above_average']
del df_test['above_average']

df_train.shape, df_val.shape, df_test.shape

((7148, 10), (2383, 10), (2383, 10))

In [16]:
df_train[categorical + numerical].head()

,make,model,transmission_type,vehicle_style,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
0,Mitsubishi,Endeavor,AUTOMATIC,4dr SUV,2011,225.0,6.0,19,15,33599
1,Kia,Borrego,AUTOMATIC,4dr SUV,2009,276.0,6.0,21,17,26245
2,Lamborghini,Gallardo,MANUAL,Convertible,2012,570.0,10.0,20,12,248000
3,Chevrolet,Colorado,AUTOMATIC,Crew Cab Pickup,2016,200.0,4.0,27,20,24990
4,Pontiac,Vibe,AUTOMATIC,4dr Hatchback,2009,158.0,4.0,26,20,20475


In [17]:
dv = DictVectorizer(sparse=False)
train_dict = df_train.to_dict(orient='records')
dv.fit(train_dict)
X_train = dv.transform(train_dict)
X_train.shape

(7148, 944)

In [18]:
val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)
X_val.shape

(2383, 944)

In [19]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [20]:
y_pred = model.predict_proba(X_val)[:, 1]
price_decision = (y_pred > 0.5)
(y_val == price_decision).mean().round(2)

1.0

In [21]:
def prepare_data_and_train(column_to_remove):
  df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
  df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

  df_full_train = df_full_train.reset_index(drop=True)
  df_train = df_train.reset_index(drop=True)
  df_val = df_val.reset_index(drop=True)
  df_test = df_test.reset_index(drop=True)

  y_train = df_train.above_average.values
  y_val = df_val.above_average.values
  y_test = df_test.above_average.values

  del df_train['above_average']
  del df_val['above_average']
  del df_test['above_average']

  del df_train[column_to_remove]
  del df_val[column_to_remove]

  dv = DictVectorizer(sparse=False)
  train_dict = df_train.to_dict(orient='records')
  dv.fit(train_dict)
  X_train = dv.transform(train_dict)

  val_dict = df_val.to_dict(orient='records')
  X_val = dv.transform(val_dict)

  model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
  model.fit(X_train, y_train)
  y_pred = model.predict_proba(X_val)[:, 1]
  price_decision = (y_pred >= 0.5)
  return (y_val == price_decision).mean()

for column in ['year', 'engine_hp', 'transmission_type', 'city_mpg']:
  accuracy = prepare_data_and_train(column)
  print(f"{column} - {str(1.0 - accuracy)}")


year - 0.05287452790600089
engine_hp - 0.0
transmission_type - 0.00041963911036513313
city_mpg - 0.00041963911036513313


In [37]:
df["price"] = np.log(df["price"])
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,10.739327
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,10.612754
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,10.500949
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,10.290449
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,10.448715


In [40]:
if "above_average" in df.columns:
  del df["above_average"]
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,10.739327
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,10.612754
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,10.500949
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,10.290449
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,10.448715


In [49]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

df_full_train = df_full_train.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.price.values
y_val = df_val.price.values
y_test = df_test.price.values

del df_train['price']
del df_val['price']
del df_test['price']

dv = DictVectorizer(sparse=True)
train_dict = df_train.to_dict(orient='records')
dv.fit(train_dict)
X_train = dv.transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [46]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return round(np.sqrt(mse), 3)

In [50]:
for a in [0, 0.01, 0.1, 1, 10]:
  model = Ridge(solver='sag', random_state=42, alpha=a)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_val)
  print(f"a: {a}, rmse: {str(rmse(y_val, y_pred))}")


a: 0, rmse: 0.255
a: 0.01, rmse: 0.255
a: 0.1, rmse: 0.255
a: 1, rmse: 0.258
a: 10, rmse: 0.336
